# Utilisation de vectorize et guvectorize

<img src="https://github.com/lsteffenel/cours_numba_2017/blob/master/figures/numba_blue_icon_rgb.png?raw=1" alt="Drawing" style="width: 20%;"/>

<center>**Loic Gouarin**</center>
<center>*8 novembre 2017*</center>

Numba fournie deux autres fonctionalités (**vectorize** et **guvectorize**) permettant d'accélérer des fonctions universelles NumPy ([ufuncs](https://docs.scipy.org/doc/numpy/reference/ufuncs.html)). Avant d'aller plus loin, rappelons ce qu'est une ufunc.

Une ufunc est une fonction qui agit élément par élément pour un tableau donné. Elle permet de vectoriser les opérations et elle est définie de la manière suivante: les paramètres d'entrée sont des scalaires et le paramètre de sortie est également un scalaire. 

Prenons par exemple la fonction heaviside qui est définie de la manière suivante

$$
heaviside(x, h_0) = 
\left\{
\begin{array}{l}
0 \; \text{si} \; x<0, \\
h_0 \; \text{si} \; x=0, \\
1 \; \text{si} \; x>0.
\end{array}
\right.
$$

Les ufuncs sont codées en C dans NumPy et sont donc optimisées. Voici la version de la fonction heaviside dans NumPy (attention, cette fonction n'est disponible qu'à partir de la dernière version: 1.13).

```C
/**begin repeat
 * #type = npy_float, npy_double, npy_longdouble#
 * #c = f, ,l#
 * #C = F, ,L#
 */

@type@ npy_heaviside@c@(@type@ x, @type@ h0)
{
    if (npy_isnan(x)) {
        return (@type@) NPY_NAN;
    }
    else if (x == 0) {
        return h0;
    }
    else if (x < 0) {
        return (@type@) 0.0;
    }
    else {
        return (@type@) 1.0;
    }
}
```

Comme vous pouvez le voir ce n'est pas très lisible et surtout, vous n'avez certainement pas envie de faire une interface C pour une fonction simple.

**vectorize** et **guvectorize** sont donc là pour vous aider à construire des ufuncs tout en restant en Python.

## vectorize

Voici comment la fonction heaviside s'écrit en Numba en utilisant **vectorize**.

In [ ]:
from numba import vectorize

@vectorize(['float64(float64, float64)'])
def heaviside(x, h0):
    if x == 0:
        return h0
    elif x < 0:
        return 0.
    else:
        return 1.

In [ ]:
import numpy as np

x = -1 + 2*np.random.random(1000000)
h0 = .4 
numba_res = heaviside(x, h0)
numpy_res = np.heaviside(x, h0)

In [ ]:
np.all(numba_res == numpy_res)

In [ ]:
%timeit heaviside(x, h0)

In [ ]:
%timeit np.heaviside(x, h0)

La fonction Numba est plus lente que la fonction NumPy. On peut accélérer un peu les choses en spécifiant la cible. En effet, **vectorize** a un mot clé **target** indiquant

- 'cpu'

fonction pour un thread sur CPU

- 'parallel'

fonction multi-threads pour CPU

- 'cuda'

fonction utilisant cuda sur GPU

Voyons ce que ça donne si nous mettons cette fonction en parallèle.

In [ ]:
@vectorize(['float64(float64, float64)'], target='parallel')
def heaviside_para(x, h0):
    if x == 0:
        return h0
    elif x < 0:
        return 0.
    else:
        return 1.

On l'appelle une première fois pour ne pas prendre en compte le temps de compilation dans le calcul de la performance.

In [ ]:
numba_res_para = heaviside_para(x, h0)
np.all(numba_res_para == numpy_res)

In [ ]:
%timeit heaviside_para(x, h0)

Nous sommes cette fois un peu plus rapide que la fonction NumPy. Mais il faut noté que la version NumPy n'est pas parallèle.

In [ ]:
@vectorize(['float64(float64, float64)'], target='cuda')
def heaviside_gpu(x, h0):
    if x == 0:
        return h0
    elif x < 0:
        return 0.
    else:
        return 1.

In [ ]:
heaviside_gpu(x, h0)

In [ ]:
%timeit heaviside_gpu(x, h0)

Comme vu lors l'introduction à **@jit**, il est possible de spécialiser une fonction vectorize en donnant les différents types sous forme de liste.

In [ ]:
@vectorize(['float64(float64, float64)',
            'float32(float32, float32)',
            'int32(int32, int32)'], target='parallel')
def heaviside_para(x, h0):
    if x == 0:
        return h0
    elif x < 0:
        return 0.
    else:
        return 1.

## guvectorize

**guvectorize** fait exactement la même chose que **vectorize** à la seule différence que nous n'avons à aucun moment spécifié le tableau de sortie. Celui-ci est donc créé à chaque appel de la fonction. Ce qui peut avoir un coup non négligeable si nous appelons plein de fois la fonction. **guvectorize** permet de mettre le tableau de sortie dans les arguments.

Reprenons notre exemple

In [ ]:
from numba import guvectorize    

@guvectorize(['(float64[:], float64[:], float64[:])'], '(),()->()', target='parallel')
def guheaviside(x, h0, y):
    if x[0] == 0:
        y[0] = h0[0]
    elif x[0] < 0:
        y[0] = 0.
    else:
        y[0] = 1.      

In [ ]:
y = np.zeros_like(x)
guheaviside(x, h0, y)
np.all(y == numpy_res)

In [ ]:
%timeit guheaviside(x, h0, y)

**guvectorize** permet de gérer plus facilement le parallèlisme que **vectorize** lorsque vous ne faites pas des opérations élément par élément mais que vous avez des formules un peu plus complexes.

Supposons que nous voulions calculer la racine carrée de la somme des éléments au carré des éléments colonnes d'une matrice. Ces calculs peuvent se faire de manière indépendante et de façon parallèle sur chacune des lignes.

Voici comment l'écrire avec **guvectorize**.

In [ ]:
from math import sqrt
from numba import njit, guvectorize
import numpy as np

@njit
def square_sum(arr):
    a = 0.
    for i in range(arr.size):
        a = sqrt(a**2 + arr[i]**2)
    return a

@guvectorize(["void(float64[:], float64[:])"], "(n) -> ()", target="parallel", nopython=True)
def row_sum_gu(input_array, output_array) :
    output_array[0] = square_sum(input_array)

@njit
def row_sum_jit(input_array, output_array) :
    m, n = input_array.shape
    for i in range(m) :
        output_array[i] = square_sum(input_array[i,:])
    return output_array


In [ ]:
rows = int(64)
columns = int(1e6)

input_array = np.random.random((rows, columns))
output_array = np.zeros((rows))
output_array2 = np.zeros((rows))

# Warmup an check that they are equal 
np.allclose(row_sum_jit(input_array, output_array), row_sum_gu(input_array, output_array2))
%timeit row_sum_jit(input_array, output_array.copy())  # 10 loops, best of 3: 130 ms per loop
%timeit row_sum_gu(input_array, output_array.copy())   # 10 loops, best of 3: 35.7 ms per loop

Le calcul du carré et de la racine carrèe ont un coût de calcul important. On voit ici tout le bénéfice d'utiliser **guvectorize** pour profiter du parallèlisme.

## Exercice

Proposez une version de cette fonction en utilisant **guvectorize**.

In [ ]:
def splint(xa, ya, y2a, x, y):
    n = xa.shape[0]
    for i in range(x.shape[0]):
        klo = 0
        khi = n-1
        while(khi-klo) > 1:
            k = (khi+klo) >> 1
            if xa[k] > x[i]:
                khi = k
            else:
                klo = k
        h = xa[khi] - xa[klo]
        a = (xa[khi]-x[i])/h    
        b = (x[i]-xa[klo])/h
        y[i,:] = a*ya[klo,:]+b*ya[khi,:]+((a**3-a)*y2a[klo,:]+(b**3-b)*y2a[khi,:])*h**2/6.